In [ ]:
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.responses import JSONResponse
from google import genai
from PIL import Image
import io
import json
import re   
import os
from dotenv import load_dotenv

In [10]:
load_dotenv()

True

In [9]:
app = FastAPI()

In [13]:
api = os.getenv('GENAI_API_KEY')
client = genai.Client(api_key=api)

In [14]:
prompt = (
    "Transcribe the nutrition facts table in this image and output the data "
    "as a single **JSON object**. Use keys like 'serving-size', 'energy-kcal', 'fat', 'carbohydrates', 'proteins', 'saturated-fat', 'trans-fat', 'sugars', 'added-sugars', 'sodium', 'salt', and 'fiber'. "
    "Do not include any text outside of the JSON object."
    "If the values are not in the image, fill the values with 0"
    "Pay attention to the unit, normalize the unit so it's stated in g (gram) instead of mg (miligram)"
)

In [ ]:
def parse_json_from_model(raw):
    cleaned = raw.strip().replace("```json", "").replace("```", "")
    cleaned = re.sub(r"\s+", " ", cleaned).strip()

    try:
        return json.loads(cleaned)
    except:
        print("Warning: Fallback JSON parser used.")
        data = {}
        body = cleaned.replace("{", "").replace("}", "")
        for pair in body.split(","):
            if ":" not in pair:
                continue
            key, value = pair.split(":", 1)
            key = key.replace('"', '').strip()
            try:
                value = float(value.replace('"', '').strip())
            except:
                value = 0
            data[key] = value
        return data

In [ ]:
@app.post("/ocr")
async def ocr_endpoint(file: UploadFile = File(...)):
    try:
        img_bytes = await file.read()
        image = Image.open(io.BytesIO(img_bytes))

        result = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=[prompt, image],
        )

        raw_json = result.text
        parsed = parse_json_from_model(raw_json)

        serving = parsed.get("serving-size", "0g")
        match = re.findall(r"([\d\.]+)", str(serving))
        divider = float(match[0]) if match else 1

        normalized = {}
        for k, v in parsed.items():
            if k == "serving-size":
                continue
            try:
                num = float(v)
            except:
                num = 0
            normalized[k + "_1g"] = round(num / divider, 3)

        return JSONResponse(normalized)

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))